<a href="https://colab.research.google.com/github/uofg-iom/CNN-Cropper/blob/main/Croppy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

ROOT = "/content/drive/My Drive/DSTP/"

In [ ]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

thing_classes = ['cheek','forehead']
thing_folder  = 'faces'

def get_labelled_dicts(directory):
    classes = thing_classes
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["height"] = img_anns["imageHeight"]
        record["width"] = img_anns["imageWidth"]
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']]
            py = [a[1] for a in anno['points']]
            poly = [(x, y) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in ["train", "test"]:
    DatasetCatalog.register(thing_folder+"_" + d, lambda d=d: get_labelled_dicts(ROOT+thing_folder+'/' + d))
    MetadataCatalog.get(thing_folder+"_" + d).set(thing_classes=thing_classes)
skins_metadata = MetadataCatalog.get(thing_folder+"_train")

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.MODEL.WEIGHTS = os.path.join("/content/drive/My Drive/DSTP/model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
cfg.DATASETS.TEST = (thing_folder+"_test", )
predictor = DefaultPredictor(cfg)

import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_labelled_dicts(ROOT+thing_folder+'/validation')
print(dataset_dicts)

for d in random.sample(dataset_dicts, 1):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=skins_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
# [list(el2) for el2 in list(el)]
boxes = [el.numpy() for el in list(vars(outputs["instances"].to("cpu"))["_fields"]["pred_boxes"])]

print(boxes)
# print(im.shape)

def crop(box,im):
  box = [int(el) for el in box]
  # print(box)
  top = box[1]
  bottom = box[3]
  left = box[0]
  right = box[2]
  return im[top:bottom,left:right,:]
  

for i,box in enumerate(boxes):
  cropped = crop(box,im)
  plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
  plt.show()
  cv2.imwrite("/content/output/cropped"+str(i)+".png",cropped)


In [ ]:
def predictAndCrop(filename,outputFolder="/content/output",outputName="cropped"):
  im = cv2.imread(filename)

  outputs = predictor(im)

  boxes = [el.numpy() for el in list(vars(outputs["instances"].to("cpu"))["_fields"]["pred_boxes"])]

  for i,box in enumerate(boxes):
    cropped = crop(box,im)
    # plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
    # plt.show()
    cv2.imwrite(os.path.join(outputFolder,outputName+str(i)+".png"),cropped)

predictAndCrop("/content/IMG_2942.png",outputName="outone")
predictAndCrop("/content/IMG_2943.png",outputName="outtwo")
predictAndCrop("/content/IMG_2944.png",outputName="outthree")

In [ ]:
predictAndCrop("/content/IMG_20230426_114746_HDR.JPG",outputName="outfour")

In [ ]:
predictAndCrop("/content/Camera shot 2023-04-26 at 13.54.33.png",outputName="outfive")

In [ ]:
predictAndCrop("/content/Camera shot 2023-04-26 at 13.57.06.png",outputName="outfive")

In [ ]:
!cp -r /content/output /content/drive/MyDrive/DSTP/croppedCheeks/